In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/full/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


47946654

In [3]:
users_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/full/allUsers/*.gz")
print users_lines
users_lines.count()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2


7978096

In [4]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime
import time


In [5]:
def parseUser(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "CreationDate" in root.attrib:
                return(root.attrib["Id"], root.attrib["CreationDate"])
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            

In [6]:
users = users_lines.map(parseUser).filter(lambda x: x!= 'Empty')
print users.count()
print users.take(20)

4016017
[('2629093', '2013-07-29T06:01:42.410'), ('2629094', '2013-07-29T06:02:00.537'), ('2629095', '2013-07-29T06:02:44.543'), ('2629096', '2013-07-29T06:04:16.690'), ('2629097', '2013-07-29T06:04:34.343'), ('2629098', '2013-07-29T06:04:58.327'), ('2629099', '2013-07-29T06:05:24.410'), ('2629100', '2013-07-29T06:05:31.910'), ('2629101', '2013-07-29T06:05:33.473'), ('2629102', '2013-07-29T06:05:49.413'), ('2629103', '2013-07-29T06:05:53.743'), ('2629104', '2013-07-29T06:06:35.443'), ('2629105', '2013-07-29T06:07:18.887'), ('2629106', '2013-07-29T06:07:30.703'), ('2629107', '2013-07-29T06:08:50.893'), ('2629108', '2013-07-29T06:08:55.783'), ('2629109', '2013-07-29T06:09:10.367'), ('2629110', '2013-07-29T06:09:39.023'), ('2629111', '2013-07-29T06:09:45.837'), ('2629112', '2013-07-29T06:09:47.430')]


In [7]:
def parsePost(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    return (root.attrib["OwnerUserId"], root.attrib["CreationDate"])
                except:
                    pass
                    return ("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [8]:
posts = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')
print posts
print posts.take(20)
posts.count()


PythonRDD[8] at RDD at PythonRDD.scala:43
[('2433934', '2014-04-22T06:15:07.070'), ('1692695', '2014-04-22T06:15:07.190'), ('2655465', '2014-04-22T06:15:33.113'), ('3189313', '2014-04-22T06:15:37.650'), ('3330929', '2014-04-22T06:15:46.763'), ('3557296', '2014-04-22T06:15:59.837'), ('692977', '2014-04-22T06:16:00.200'), ('3557098', '2014-04-22T06:16:03.460'), ('2616474', '2014-04-22T06:16:04.017'), ('2020444', '2014-04-22T06:16:05.267'), ('133802', '2014-04-22T06:16:11.333'), ('418864', '2014-04-22T06:16:11.633'), ('2846794', '2014-04-22T06:16:12.743'), ('1915323', '2014-04-22T06:16:12.957'), ('2541124', '2014-04-22T06:16:15.587'), ('1060208', '2014-04-22T06:16:19.550'), ('2156712', '2014-04-22T06:16:20.430'), ('2504224', '2014-04-22T06:16:21.340'), ('1446916', '2014-04-22T06:16:22.707'), ('3319147', '2014-04-22T06:16:26.467')]


23875001

In [9]:
def parseQuestion(line):
    if '<row'in line:
        try:
            root = ET.fromstring(line.encode('utf8').strip())
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "OwnerUserId" in root.attrib:
                p_type = root.attrib["PostTypeId"]
                owner = root.attrib["OwnerUserId"]
                if p_type == '1':
                    score = float(root.attrib["Score"])
                    views = float(root.attrib["ViewCount"])
                    answers = float(root.attrib["AnswerCount"])
                    create_date = root.attrib["CreationDate"] 
                    favs = 0.
                    try:
                        favs = float(root.attrib["FavoriteCount"])
                    except:
                        pass

                    return(owner, [create_date, score, views, answers, favs])
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [10]:
questions = posts_lines.map(parseQuestion).filter(lambda x: x!= "Empty")\
            .reduceByKey(min).map(lambda x: (x[0],x[1][1:]))
print questions.count()
print questions.take(20)

1664063
[('400303', [0.0, 861.0, 1.0, 0.0]), ('3233915', [0.0, 63.0, 1.0, 0.0]), ('1394056', [1.0, 12115.0, 3.0, 0.0]), ('1988847', [1.0, 1832.0, 1.0, 0.0]), ('3382688', [1.0, 195.0, 1.0, 0.0]), ('4022777', [-2.0, 28.0, 2.0, 0.0]), ('4227558', [0.0, 16.0, 1.0, 0.0]), ('656672', [3.0, 450.0, 1.0, 0.0]), ('382512', [5.0, 13683.0, 3.0, 0.0]), ('2811225', [0.0, 679.0, 1.0, 0.0]), ('4140206', [0.0, 213.0, 1.0, 0.0]), ('2925687', [1.0, 131.0, 5.0, 0.0]), ('2140507', [0.0, 313.0, 4.0, 0.0]), ('2774785', [0.0, 112.0, 1.0, 0.0]), ('3776714', [1.0, 47.0, 1.0, 0.0]), ('3637553', [0.0, 223.0, 1.0, 0.0]), ('2567188', [1.0, 332.0, 4.0, 0.0]), ('4114752', [-2.0, 68.0, 2.0, 0.0]), ('1982978', [2.0, 306.0, 1.0, 0.0]), ('3336989', [1.0, 158.0, 1.0, 0.0])]


In [11]:
from datetime import datetime
import time

def dateConversion (date1, date2):  
    date1_obj = datetime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    date2_obj = datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = date2_obj - date1_obj
    diff_sec = diff.total_seconds()
    if diff_sec > 8640000 and diff_sec < 12960000:
        return 1
    else:
        return 0
  

In [12]:
def transform(x):
    if x[1]>=1:
        vet =1
    else:
        vet = 0
    return (x[0],vet)

In [13]:
joint = users.join(posts).map(lambda x: (x[0], dateConversion(x[1][0], x[1][1])))\
        .reduceByKey(lambda x,y: x+y).map(transform)
print joint.count()
print joint.take(40)

2134402
[('4244030', 0), ('3572161', 0), ('2366235', 0), ('3081058', 0), ('4517024', 0), ('226394', 0), ('3415433', 0), ('2067486', 0), ('2089439', 0), ('4533468', 0), ('4245037', 0), ('3172408', 1), ('49114', 0), ('2277308', 0), ('945718', 0), ('3808011', 0), ('2279515', 0), ('2171253', 0), ('1025276', 0), ('2528105', 1), ('3023252', 0), ('3271310', 0), ('2062661', 0), ('3433536', 0), ('2052956', 0), ('1096793', 0), ('472777', 0), ('904209', 0), ('3358938', 0), ('1834308', 0), ('1518873', 0), ('1554668', 1), ('2430620', 1), ('4569777', 0), ('3551380', 0), ('4546220', 0), ('794996', 1), ('2742088', 0), ('2005020', 0), ('3927882', 0)]


In [14]:
veterans = joint.join(questions).map(lambda x: (x[1][0], x[1][1]+[1]))\
            .reduceByKey(lambda x,y: np.add(x,y))\
            .map(lambda x: (x[0], [x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]))
print veterans.count()
print veterans.take(40)

2
[(0, [1.1307456144103445, 1096.1519220732553, 1.5038565525030159, 0.38617644458514078]), (1, [2.2598437331442924, 1844.0344896669696, 1.8426197044183144, 0.86731572377444555])]
